# Checkpointing PyTrees of Arrays

A [PyTree](https://jax.readthedocs.io/en/latest/pytrees.html) is the most common way of representing a training state in JAX. While Orbax is designed to be as generic as possible, and provides customization options for all manner of checkpointable objects, PyTrees naturally have pride of place. Furthermore, the standard object used to represent large, sharded arrays is the `jax.Array`. This, too, has extensive first-class support.

## `CheckpointHandler` Support

There are essentially two options provided by Orbax for working with PyTrees.


*   {py:class}`StandardCheckpointHandler <orbax.checkpoint.StandardCheckpointHandler>` - applicable in the **majority** of use cases.
*   {py:class}`PyTreeCheckpointHandler <orbax.checkpoint.PyTreeCheckpointHandler>` - useful when advanced customization is desired.



In [34]:
import numpy as np
import orbax.checkpoint as ocp
import jax

In [35]:
sharding = jax.sharding.NamedSharding(
    jax.sharding.Mesh(jax.devices(), ('model',)),
    jax.sharding.PartitionSpec(
        'model',
    ),
)
create_sharded_array = lambda x: jax.device_put(x, sharding)
state = {
    'a': np.arange(16),
    'b': np.ones(16),
}
state = jax.tree_util.tree_map(create_sharded_array, state)
abstract_state = jax.tree_util.tree_map(ocp.utils.to_shape_dtype_struct, state)

## Basic Checkpointing

Let's use `StandardCheckpointHandler` to work with PyTrees of `jax.Array`.

In [36]:
path = ocp.test_utils.erase_and_create_empty('/tmp/basic/')
# Make sure to use async for improved performance!
ckptr = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())

In [37]:
ckptr.save(path / '1', args=ocp.args.StandardSave(state))
ckptr.wait_until_finished()

We specify the `abstract_state` in order to restore with the given dtypes, shapes, and shardings for each leaf.

In [38]:
restored = ckptr.restore(path / '1', args=ocp.args.StandardRestore(abstract_state))

In [39]:
restored

In [40]:
restored['a'].sharding

You can do the exact same with a "concrete" target rather than an "abstract" target. However, this requires that you fully initialize the target train state
before restoring from the checkpoint, which is inefficient. It is better practice to only initialize metadata (either by manually creating `jax.ShapeDtypeStruct`s or using `jax.eval_shape`).

In [41]:
ckptr.restore(path / '1', args=ocp.args.StandardRestore(state))

### Customizing Restored Properties

#### Array dtype

In [42]:
def set_restore_dtype(x: jax.ShapeDtypeStruct) -> jax.ShapeDtypeStruct:
  return x.update(dtype=np.int16)

cast_dtype_abstract_state = jax.tree_util.tree_map(
    set_restore_dtype, abstract_state)

In [43]:
ckptr.restore(
    path / '1',
    args=ocp.args.StandardRestore(cast_dtype_abstract_state),
)

#### Pad / truncate shape

In [44]:
different_shape_abstract_state = {
    'a': jax.ShapeDtypeStruct(
        shape=(8,),
        dtype=abstract_state['a'].dtype,
        sharding=abstract_state['a'].sharding
    ),
    'b': jax.ShapeDtypeStruct(
        shape=(32,),
        dtype=abstract_state['b'].dtype,
        sharding=abstract_state['b'].sharding
    ),
}

Ordinarily, specifying a target array with a different shape than in the
checkpoint results in an error.

In [45]:
try:
  ckptr.restore(
      path / '1',
      args=ocp.args.StandardRestore(different_shape_abstract_state),
  )
except BaseException as e:
  print(e)

We can pad or truncate arrays as they are loaded by specifying `strict=False`.

In [46]:
ckptr.restore(
    path / '1',
    args=ocp.args.StandardRestore(different_shape_abstract_state, strict=False),
)

#### Change sharding

**NOTE: This can often be a particularly sharp edge.**

Sharding commonly needs to be changed when loading a checkpoint saved on one topology to a different topology.

**If changing topologies, you MUST specify sharding when restoring.**

Unless you are loading on the exact same topology, Orbax does not make any decisions about shardings on you behalf. If you have the exact same topology,
however, it is possible to avoid specifying the sharding when loading. This is demonstrated below:

In [47]:
restored = ckptr.restore(path / '1')

In [48]:
restored['a'].sharding

In the example below, we alter the sharding while loading.

In [49]:
sharding = jax.sharding.NamedSharding(
    jax.sharding.Mesh(jax.devices(), ('x',)),
    jax.sharding.PartitionSpec(),
)
def set_sharding(x: jax.ShapeDtypeStruct) -> jax.ShapeDtypeStruct:
  return x.update(sharding=sharding)

change_sharding_abstract_state = jax.tree_util.tree_map(
    set_sharding, abstract_state)
restored = ckptr.restore(
    path / '1',
    args=ocp.args.StandardRestore(change_sharding_abstract_state),
)

In [50]:
restored['a'].sharding

(pytrees-partial-restore-main)=
### Partial Restore

You may wish to restore part of a PyTree contained within a saved checkpoint. For example, consider the following item:

In [51]:
original_item = {
    'params': {
        'layer1': {
            'kernel': np.arange(8),
            'bias': np.arange(8),
        },
        'layer2': {
            'kernel': np.arange(8),
            'bias': np.arange(8),
        },
    },
    'opt_state': [np.arange(8), np.arange(8)],
    'step': 101,
}

path = ocp.test_utils.erase_and_create_empty('/tmp/partial/')
ckptr = ocp.PyTreeCheckpointer()
ckptr.save(path / '1', args=ocp.args.PyTreeSave(original_item))

If we want to restore only a subset of PyTree nodes (`params.layer2` and `step`, for example), we can use Placeholder values.

#### Placeholder

To restore part of a PyTree item, we can specify which nodes to ignore during restoration by using `ocp.PLACEHOLDER`.

In [52]:
reference_item = {
    'params': {
        'layer1': {
            'kernel': ocp.PLACEHOLDER,
            'bias': ocp.PLACEHOLDER,
        },
        'layer2': {
            'kernel': np.arange(8),
            'bias': np.arange(8),
        },
    },
    'opt_state': [ocp.PLACEHOLDER, ocp.PLACEHOLDER],
    'step': 101,
}

ckptr.restore(
    path / '1',
    args=ocp.args.PyTreeRestore(
        item=reference_item,
    ),
)

#### Omission

Alternatively, we can enable the `partial_restore` option to avoid having to explicitly specify nodes to be skipped. Instead, we simply ignore those nodes during construction of the abstract PyTree.

In [53]:
reference_item = {
    'params': {
        # Note: omit 'layer1' to avoid loading it (or, more accurately, to avoid
        # loading 'layer1.kernel' and 'layer1.bias')
        'layer2': {
            'kernel': np.arange(8),
            'bias': np.arange(8),
        },
    },
    # Omit 'opt_state' to avoid loading 'opt_state[0]' and 'opt_state[1]'
    'step': 101,
}

# Loading PyTrees with certain leaves missing is unsafe
try:
  ckptr.restore(
      path / '1',
      args=ocp.args.PyTreeRestore(
          item=reference_item,
      ),
  )
except ValueError as e:
  print(e)

# So Partial Restore Omission mode must be opted-into using partial_restore=True
ckptr.restore(
    path / '1',
    args=ocp.args.PyTreeRestore(
        item=reference_item,
        partial_restore=True,
    ),
)

## Advanced Options

There are some advanced options that `StandardCheckpointHandler` does not provide. Additional options can be specified using `PyTreeCheckpointHandler`
instead.

### Saving

For example, `PyTreeCheckpointHandler` can be used to customize the on-disk type used to save individual arrays. First, let's save and restore as normal.

In [54]:
path = ocp.test_utils.erase_and_create_empty('/tmp/advanced/')
# Make sure to use async for improved performance!
ckptr = ocp.AsyncCheckpointer(ocp.PyTreeCheckpointHandler())

In [55]:
ckptr.save(path / '1', args=ocp.args.PyTreeSave(state))
ckptr.wait_until_finished()

In [56]:
restored = ckptr.restore(path / '1')

In [57]:
restored['a'].dtype

In [58]:
restored['b'].dtype

Now, let's set the dtype of the array when saving.

In [59]:
ckptr.save(
    path / '2',
    args=ocp.args.PyTreeSave(
        state,
        save_args={
          # We must set one ocp.SaveArgs per leaf.
          'a': ocp.SaveArgs(dtype=np.dtype(np.int16)),
          'b': ocp.SaveArgs()
        }
    ),
)
ckptr.wait_until_finished()

In [60]:
restored = ckptr.restore(path / '2')

In [61]:
restored['a'].dtype

In [62]:
restored['b'].dtype

### Restoring

Options similar to the above are available, where we can customize shape, dtype, and sharding when restoring.

In [63]:
ckptr.restore(
    path / '2',
    args=ocp.args.PyTreeRestore(
        restore_args={
          # RestoreArgs is the parent class for ArrayRestoreArgs.
          # We must set one RestoreArgs per leaf.
          'a': ocp.RestoreArgs(restore_type=np.ndarray),
          'b': ocp.ArrayRestoreArgs(dtype=np.dtype(np.int16), sharding=sharding)
        }
    ),
)

Note that "a" was restored as `np.ndarray` rather than `jax.Array`.

`PyTreeCheckpointHandler` also provides options to perform transformations when restoring via the `transforms` argument. This is useful when your target tree has a different structure than your checkpoint tree. For example, it can be used to rename keys from the checkpoint (`original_key`) to match the target structure, as seen in the example below.

In [64]:
ckptr.restore(
    path / '2',
    args=ocp.args.PyTreeRestore(
        # `item` serves as a guide to what the result tree structure should look
        # like.
        item={
            # Value doesn't really matter here, as long as it's not None.
            'c': ...,
            # Can add in extra keys.
            'd': np.arange(8)
        },
        # `restore_args` must be relative to the result tree, not the
        # checkpoint.
        restore_args={
          'c': ocp.RestoreArgs(restore_type=np.ndarray),
        },
        transforms={
            'c': ocp.Transform(original_key='a')
        },
    ),
)